In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random


In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


In [6]:
import torch.utils.data as Data
from torch import nn

In [4]:
batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [5]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 1.3210, -0.3892],
        [-0.7553, -1.3834],
        [-1.4129, -0.0197],
        [ 0.7244,  0.5818],
        [-0.6033,  1.1643],
        [ 0.9347, -0.3813],
        [ 0.5341, -1.0022],
        [-0.2598, -0.4047],
        [ 0.2260,  0.5258],
        [-0.6370,  0.4530]]) tensor([ 8.1660,  7.3745,  1.4343,  3.6801, -0.9746,  7.3708,  8.6883,  5.0660,
         2.8567,  1.3613])


In [7]:
# way 0
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [8]:
# way 1
net1 = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # ...
)

# way 2
net2 = nn.Sequential()
net2.add_module('linear', nn.Linear(num_inputs,1))
#net2.add_module

# way 3
from collections import OrderedDict
net3 = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    #, ....
]))

print(net2)


Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0571, 0.5264]], requires_grad=True)
Parameter containing:
tensor([-0.1121], requires_grad=True)


In [18]:
from torch.nn import init

# init.normal_(net1[0].weight, mean=0, std=0.01)
# init.constant_(net1[0].bias, val=0)
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

In [19]:
loss = nn.MSELoss()

In [21]:
from torch import optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [22]:
# optimizer_fineTuing = optim.SGD([
#     {'params':net.subnet1.parameters() }, # lr=0.03
#     {'params':net.subnet2.parameters(), 'lr':0.01}
# ], lr=0.03)

In [ ]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

In [23]:
num_epochs = 3

for epoch in range(1, num_epochs+1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d , loss: %f '%(epoch, l.item()))

epoch 1 , loss: 0.000392 
epoch 2 , loss: 0.000051 
epoch 3 , loss: 0.000121 


In [24]:
true_w, net.linear.weight

([2, -3.4], Parameter containing:
 tensor([[ 1.9998, -3.4000]], requires_grad=True))

In [25]:

true_b, net.linear.bias

(4.2, Parameter containing:
 tensor([4.1998], requires_grad=True))